In [ ]:
save_path = "/home/jupyter-ayoub/MEM/batches"  # Pas dit aan
os.makedirs(save_path, exist_ok=True)

batch_size = 10
total_time = rearranged_10.sizes["time"]

for i in range(0, total_time, batch_size):
    #print(f"Processing time slice {i}:{i+batch_size}...")

    batch = rearranged_10.isel(time=slice(i, min(i+batch_size, total_time)))
    batch_np = batch.values.astype(np.float32)
    batch_tensor = torch.from_numpy(batch_np)

    # Opslaan naar bestand (bv. batch_000.pt)
    torch.save(batch_tensor, os.path.join(save_path, f"fcs_batch_{i:04d}.pt"))

In [ ]:
import torch
import glob

files = sorted(glob.glob("/home/jupyter-ayoub/MEM/batches/fcs_batch_*.pt"))
fcs_whole_tensor = None

for i in range(0, len(files), 5):  # per 5 bestanden
    batch_group = [torch.load(f) for f in files[i:i+5]]
    partial = torch.cat(batch_group, dim=0)

    if fcs_whole_tensor is None:
        fcs_whole_tensor = partial
    else:
        fcs_whole_tensor = torch.cat([fcs_whole_tensor, partial], dim=0)

    # geheugen vrijmaken
    del batch_group, partial
    torch.cuda.empty_cache()  # indien nodig

print("Totale shape:", fcs_whole_tensor.shape)

In [ ]:
torch.save(fcs_whole_tensor, "/home/jupyter-ayoub/MEM/fcs_whole_tensor.pt")

In [ ]:
fcs_whole_tensor = torch.load("/home/jupyter-ayoub/MEM/fcs_whole_tensor.pt")

In [ ]:
batch_size = 100
total_time = fcs_10.sizes["time"]
save_path = "/home/jupyter-ayoub/MEM/batches_unnorm"
os.makedirs(save_path, exist_ok=True)

for i in range(0, total_time, batch_size):
    print(f"Processing time slice {i}:{i+batch_size}")
    
    batch = fcs_10.isel(time=slice(i, i+batch_size))  # selecteer klein blok
    fcs_array = batch.to_array(dim="variable")
    
    # Zet om naar float32 en PyTorch tensor
    fcs_np = fcs_array.transpose('time', 'number', 'step', 'latitude', 'longitude', 'variable').values.astype(np.float32)
    fcs_tensor = torch.from_numpy(fcs_np)
    
    torch.save(fcs_tensor, f"{save_path}/fcs_unnorm_batch_{i:04d}.pt")

In [ ]:
import torch
import glob
import os

# Pad naar je ongenormaliseerde batches
files = sorted(glob.glob("/home/jupyter-ayoub/MEM/batches_unnorm/fcs_unnorm_batch_*.pt"))

# Beginwaarde
fcs_whole_notnorm_tensor = None

# Voeg samen in stukken van 5 bestanden
for i in range(0, len(files), 5):
    print(f"Loading batches {i} to {i+4}")

    batch_group = [torch.load(f) for f in files[i:i+5]]
    partial = torch.cat(batch_group, dim=0)

    if fcs_whole_notnorm_tensor is None:
        fcs_whole_notnorm_tensor = partial
    else:
        fcs_whole_notnorm_tensor = torch.cat([fcs_whole_notnorm_tensor, partial], dim=0)

    # RAM opschonen
    del batch_group, partial
    torch.cuda.empty_cache()  # als je op GPU zit, anders niet nodig

print("Totale shape fcs_whole_notnorm_tensor:", fcs_whole_notnorm_tensor.shape)

In [ ]:
fcs_whole_notnorm_tensor = fcs_whole_notnorm_tensor.to(torch.float32)
torch.save(fcs_whole_notnorm_tensor, "/home/jupyter-ayoub/MEM/fcs_whole_notnorm_tensor.pt")

In [ ]:
fcs_whole_notnorm_tensor = torch.load("/home/jupyter-ayoub/MEM/fcs_whole_notnorm_tensor.pt")

In [ ]:
ensemble_one = postpro_10_whole[...,0]
#print(fcs_whole_notnorm_tensor)
values_tar = fcs_whole_notnorm_tensor[:,:,:,:,:,5] #torch.Size([26, 11, 32, 33])
#print(values_tar)
scale_std, scale_mean= torch.std_mean(values_tar, dim=1, unbiased=True)
scaled_ensemble_one=(ensemble_one * scale_std.unsqueeze(1) + scale_mean.unsqueeze(1)).permute(1, 0, 2, 3, 4)
#print(scaled_ensemble_one)

In [3]:
import torch
import numpy

In [4]:
path = "/home/jupyter-ayoub/results/Transformerweights/ssrd6/Beste_result/epochs50predictors18CRPSKERNELSTEPlambda0.02k3.3.pth"
with torch.serialization.safe_globals([numpy.core.multiarray.scalar]):
    data = torch.load(path, map_location=torch.device('cpu'), weights_only=False)

/tmp/ipykernel_431061/1740415332.py:2: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  with torch.serialization.safe_globals([numpy.core.multiarray.scalar]):


In [5]:
print(type(data))        # Is het een dict, een model, of iets anders?
if isinstance(data, dict):
    print(data.keys())   # Geeft je een overzicht van de onderdelen (bv. model, optimizer, epoch)

<class 'dict'>
dict_keys(['model', 'crps', 'epoch'])


In [6]:
# Stel dat het modelparameters zijn opgeslagen onder "model_state_dict":
model_weights = data.get("model_state_dict", None)
if model_weights:
    for k, v in model_weights.items():
        print(k, v.shape)  # Bekijk layer-namen en tensor shapes

In [ ]:
def compute_solar_power_from_ssrd6(ssrd6_data, efficiency=0.18, panel_area=1000.0):
    """
    Zet ssrd6 (J/m² over 6 uur) om naar geschat vermogen (W) van een zonne-energiesysteem.
    Werkt op een numpy array of xarray DataArray.

    Parameters:
    - ssrd6_data: ndarray met shape (ensemble, time, lat, lon)
    - efficiency: float, systeemefficiëntie (typisch 0.15–0.20)
    - panel_area: float, totale paneeloppervlakte in m²

    Returns:
    - power_output: zelfde shape als input, vermogen in watt
    """
    irradiance_avg = ssrd6_data / (6 * 3600)  # W/m² gemiddeld over 6 uur
    power_output = efficiency * irradiance_avg * panel_area  # W

    return power_output

In [ ]:
# Stel: Tformer_one.shape = (11, 8, 32, 33)
solar_power_output = compute_solar_power_from_ssrd6(
    Tformer_one,
    efficiency=0.19,     # bijvoorbeeld voor moderne panelen
    panel_area=10000.0   # stel dat je 10.000 m² PV-oppervlak modelleert
)